In [1]:
#GTFS API

In [2]:
%sh

pip install gtfs-realtime-bindings

Requirement already satisfied: gtfs-realtime-bindings in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (0.0.6)
Requirement already satisfied: protobuf in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from gtfs-realtime-bindings) (3.11.4)
Requirement already satisfied: setuptools in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from gtfs-realtime-bindings) (45.2.0.post20200210)
Requirement already satisfied: six>=1.9 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from protobuf->gtfs-realtime-bindings) (1.14.0)

In [3]:
spark.conf.set("fs.azure.sas.mtamnrhackathon.mtamnrdatastore.blob.core.windows.net", "?sv=2019-10-10&ss=b&srt=sco&sp=rlx&se=2020-07-25T06:48:17Z&st=2020-07-01T22:48:17Z&spr=https&sig=6FXVZ5hZ6%2F2UZsK76c8EXfoEtbUdko%2BbK8BGSXYgoMI%3D")

In [4]:
def split_map(df,col):
  keys = (dataDF
    .select(explode(col))
    .select("key")
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect())

  exprs = [F.col(col).getItem(k).alias(k) for k in keys]
  return exprs

In [5]:
def del_none(d):
    """
    Delete keys with the value ``None`` in a dictionary, recursively.

    This alters the input so you may wish to ``copy`` the dict first.
    """
    # For Python 3, write `list(d.items())`; `d.items()` won’t work
    # For Python 2, write `d.items()`; `d.iteritems()` won’t work
    for key, value in list(d.items()):
        if value is None:
            del d[key]
        elif isinstance(value, dict):
            del_none(value)
    return d 

In [6]:
from google.transit import gtfs_realtime_pb2
from google.protobuf import json_format
import json
from datetime import datetime
import pytz
import urllib
from pyspark.sql.functions import lit
from pyspark.sql import functions as F

tz_NY = pytz.timezone('America/New_York')
datetime_NY = datetime.now(tz_NY)

current_time = int(datetime_NY.strftime("%H%M"))

feed = gtfs_realtime_pb2.FeedMessage()
r = urllib.request.Request('https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/mnr%2Fgtfs-mnr', headers={"x-api-key":"enikHqkchS3ADUlcsW2XY4BpTBVhCim9Kp1D351e"})
response = urllib.request.urlopen(r)
feed.ParseFromString(response.read())

json_data= json.loads(json_format.MessageToJson(feed))

ts = json_data['header']['timestamp']

dataDF = spark.createDataFrame(json_data['entity'])
exprs1 = split_map(dataDF,"tripUpdate")
dataDF =  dataDF.select('id',*exprs1,'vehicle')
exprs1 = split_map(dataDF,"trip")
exprs2 = split_map(dataDF,"vehicle")
dataDF =  dataDF.select('id','stopTimeUpdate',*exprs1,*exprs2)
exprs1 = split_map(dataDF,"position")
exprs2 = split_map(dataDF,"vehicle")
exprs3 = split_map(dataDF,"trip")
dataDF =  dataDF.select('id','stopTimeUpdate','startTime','routeId','startDate','currentStatus','stopId','timestamp',*exprs1,*exprs2,*exprs3)

data_GTFS = dataDF.withColumn('timestamp', lit(ts))


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-907950632236046> in <module> 
 23 
 24 dataDF = spark . createDataFrame ( json_data [ 'entity' ] ) 
 ---> 25 exprs1 = split_map ( dataDF , "tripUpdate" ) 
 26 dataDF = dataDF . select ( 'id' , * exprs1 , 'vehicle' ) 
 27 exprs1 = split_map ( dataDF , "trip" ) 

 <command-907950632236050> in split_map (df, col) 
 1 def split_map ( df , col ) : 
 2 keys = (dataDF
 ----> 3 . select ( explode ( col ) ) 
 4 . select ( "key" ) 
 5 . distinct ( ) 

 NameError : name 'explode' is not defined

In [7]:
display(data_GTFS)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-907950632236056> in <module> 
 ----> 1 display ( data_GTFS ) 

 NameError : name 'data_GTFS' is not defined

In [8]:
feed = gtfs_realtime_pb2.FeedMessage()

r = urllib.request.Request('https://mnorthstg.prod.acquia-sites.com/wse/MYmta/Trains/v4/1/33/0/2020/07/15/0000/9de8f3b1-1701-4229-8ebc-346914043f4a/TripStatus24')
response = urllib.request.urlopen(r)

json_data = json.loads(response.read())

data_list  = json_data['GetTripStatusJsonResult']
data = []

for i in data_list:
    data.append(del_none(i))
dataDF_train = spark.createDataFrame(data)
display(dataDF)

id,tripUpdate,vehicle
3248,"Map(stopTimeUpdate -> null, trip -> Map(startTime -> 1701, routeId -> 3, startDate -> 07152020))","Map(trip -> Map(tripId -> 3248), vehicle -> Map(label -> 3248), timestamp -> null)"
3348,"Map(stopTimeUpdate -> null, trip -> Map(startTime -> 1655, routeId -> 3, startDate -> 07152020))","Map(trip -> Map(tripId -> 3348), vehicle -> Map(label -> 3348), timestamp -> null)"
3350,"Map(stopTimeUpdate -> null, trip -> Map(startTime -> 1730, routeId -> 3, startDate -> 07152020))","Map(trip -> Map(tripId -> 3350), vehicle -> Map(label -> 3350), timestamp -> null)"
3546,"Map(stopTimeUpdate -> null, trip -> Map(startTime -> 1659, routeId -> 3, startDate -> 07152020))","Map(trip -> Map(tripId -> 3546), vehicle -> Map(label -> 3546), timestamp -> null)"
3548,"Map(stopTimeUpdate -> null, trip -> Map(startTime -> 1715, routeId -> 3, startDate -> 07152020))","Map(trip -> Map(tripId -> 3548), vehicle -> Map(label -> 3548), timestamp -> null)"
3550,"Map(stopTimeUpdate -> null, trip -> Map(startTime -> 1759, routeId -> 3, startDate -> 07152020))","Map(trip -> Map(tripId -> 3550), vehicle -> Map(label -> 3550), timestamp -> null)"
3840,"Map(stopTimeUpdate -> null, trip -> Map(startTime -> 1550, routeId -> 5, startDate -> 07152020))","Map(stopId -> null, timestamp -> null, vehicle -> Map(label -> 3840), trip -> Map(tripId -> 3840), position -> Map(latitude -> 41.0770149230957, longitude -> -73.47164916992188), currentStatus -> null)"
3856,"Map(stopTimeUpdate -> null, trip -> Map(startTime -> 2045, routeId -> 5, startDate -> 07152020))","Map(trip -> Map(tripId -> 3856), vehicle -> Map(label -> 3856), timestamp -> null)"
4349,"Map(stopTimeUpdate -> null, trip -> Map(startTime -> 1723, routeId -> 2, startDate -> 07152020))","Map(trip -> Map(tripId -> 4349), vehicle -> Map(label -> 4349), timestamp -> null)"
4547,"Map(stopTimeUpdate -> null, trip -> Map(startTime -> 1632, routeId -> 2, startDate -> 07152020))","Map(stopId -> null, timestamp -> null, vehicle -> Map(label -> 4547), trip -> Map(tripId -> 4547), position -> Map(latitude -> 40.82160186767578, longitude -> -73.92152404785156), currentStatus -> null)"


In [9]:
feed = gtfs_realtime_pb2.FeedMessage()

r = urllib.request.Request('https://mnorthstg.prod.acquia-sites.com/wse/Vtts/v2/api/trains/9de8f3b1-1701-4229-8ebc-346914043f4a/')
response = urllib.request.urlopen(r)

json_data = json.loads(response.read())

data_list  = json_data['GetTripStatusJsonResult']
data = []

for i in data_list:
    data.append(del_none(i))
dataDF_train = spark.createDataFrame(data)
display(dataDF)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-3650483693578851> in <module> 
 6 json_data = json . loads ( response . read ( ) ) 
 7 
 ----> 8 data_list = json_data [ 'GetTripStatusJsonResult' ] 
 9 data = [ ] 
 10 

 TypeError : list indices must be integers or slices, not str